In [1]:
import json
import os
import random
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import accuracy_score,log_loss

Read `channel-based` annotation to a python dictionary

In [2]:
training_data_path = "../bipolar_eeg_dataset/train_filtred_channel_based.json"
testing_data_path = "../bipolar_eeg_dataset/dev_filtred_channel_based.json"

In [3]:
with open(training_data_path,'r') as f:
    training_data = json.load(f)

with open(testing_data_path,'r') as f:
    testing_data = json.load(f)

## Data generator

In [4]:
import numpy as np
import torch
import torchaudio

## XGBoost

In [5]:
# Set the tree_method parameter to gpu_hist
params = {'tree_method': 'gpu_hist'}
# Set the gpu_id parameter to the ID of your GPU
params['gpu_id'] = 0

xgb_clf = xgb.XGBClassifier(tree_method='gpu_hist',
                            learning_rate=1e-3,
                            nthread=4,
                            max_depth=10,
                            eval_metric='logloss',
                            objective="binary:logistic")

In [6]:
from sklearn.preprocessing import LabelEncoder
import sys
sys.path.append("..")
from utils.custom_loader import XGBLoader

In [43]:
test_loader = XGBLoader(testing_data[1],window_size=1)

In [44]:
x_t = None
y_t = None
for x,y in test_loader:
  if y.sum() != 0:
    x_t,y_t = x,y
    break

In [9]:
y_t

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 1.])

## Evaluate trained XGBoost.

1. `Benchmark`: No preprocessing of signal

In [ ]:
benchmark_model_file = "../utils/xgb_model_n_300_lr_0.02.json"
os.path.exists(benchmark_model_file)

model = xgb.Booster()

#model.set_param({"tree_method":"gpu_hist","max_depth":10})
model.load_model(benchmark_model_file)

In [ ]:
x_t.shape

(22, 7)

In [ ]:
preds = model.predict(xgb.DMatrix(x_t))

In [ ]:
(preds > 0.5).astype(np.uint8)

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
      dtype=uint8)

In [ ]:
y_t.astype(np.uint8)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
      dtype=uint8)

In [7]:
def get_data(dataset: list, window: int = 1):
    x = []
    y = []
    x_19 = []
    y_19 = []

    bar = tqdm(dataset, total=len(dataset), desc="Generating Matrix")

    for i, eeg_annot in enumerate(bar):
        montage = eeg_annot["montage"]
        dg = XGBLoader(eeg_annot, window_size=window)
        j = 0
        for _x, _y in dg:
            if _y.sum() == 0:
                continue
            if np.isnan(_x).sum() == 0:
                if montage not in ["01_tcp_ar", "02_tcp_le"]:
                    x_19.append(_x)  # _x[[3, 5, 12, 13], :])
                    y_19.append(_y)  # _y[[3, 5, 12, 13]])
                    continue
                x.append(_x)  # _x[[3, 5, 14, 15], :])
                y.append(_y)  # _y[[3, 5, 14, 15]])

    x_np = np.array(x)
    y_np = np.array(y)

    x_np_19 = np.array(x_19)
    y_np_19 = np.array(y_19)

    x_np = x_np.reshape(-1, x_np.shape[-1])
    y_np = y_np.reshape(-1)

    x_np_19 = x_np_19.reshape(-1, x_np_19.shape[-1])
    y_np_19 = y_np_19.reshape(-1)

    x_np = np.concatenate((x_np, x_np_19), axis=0)
    y_np = np.concatenate((y_np, y_np_19), axis=0)

    return x_np, y_np

In [9]:
x_,y_ = get_data(training_data)

Generating Matrix: 100%|██████████| 723/723 [10:46<00:00,  1.12it/s]


In [10]:
y_.shape

(3473362,)

In [19]:
len(y_[y_ == 1]) / len(y_)

0.49112358573624054

In [20]:
len(y_[y_ == 0])/ len(y_)

0.5088764142637594